In [0]:
# https://www.datacamp.com/courses/ensemble-methods-in-python

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [0]:
from sklearn.tree import DecisionTreeRegressor

**Course Description**

Continue your machine learning journey by diving into the wonderful world of ensemble learning methods! These are an exciting class of machine learning techniques that combine multiple individual algorithms to boost performance and solve complex problems at scale across different industries. Ensemble techniques regularly win online machine learning competitions as well! In this course, you’ll learn all about these advanced ensemble techniques, such as bagging, boosting, and stacking. You’ll apply them to real-world datasets using cutting edge Python machine learning libraries such as scikit-learn, XGBoost, CatBoost, and mlxtend.

## 1. Combining Multiple Models

Do you struggle to determine which of the models you built is the best for your problem? You should give up on that, and use them all instead! In this chapter, you'll learn how to **combine multiple models into one using "Voting" and "Averaging"**. You'll use these to predict the ratings of apps on the Google Play Store, whether or not a Pokémon is legendary, and which characters are going to die in Game of Thrones!

#### Predicting the rating of an app

Having explored the Google apps dataset in the previous exercise, let's now build a model that predicts the rating of an app given a subset of its features.

To do this, you'll use scikit-learn's DecisionTreeRegressor. As decision trees are the building blocks of many ensemble models, refreshing your memory of how they work will serve you well throughout this course.

We'll use the MAE (mean absolute error) as the evaluation metric. This metric is highly interpretable, as it represents the average absolute difference between actual and predicted ratings.

In [0]:
# Split into train (80%) and test(20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instantiate the regressor
reg_dt = DecisionTreeRegressor(min_samples_leaf=3, min_samples_split=9, random_state=500)

# Fit to the training set
reg_dt.fit(X_train, y_train)

# Evaluate the performance of the model on the test set
y_pred = reg_dt.predict(X_test)
print('MAE: {:.3f}'.format(mean_absolute_error(y_test, y_pred)))

**Observation:**

As the MAE is around 0.61, the error of the model is, on average, less than one rating point away from the actual value! Having refreshed our understanding of how to build scikit-learn models, let's now jump into our first ensemble method: Voting!

#### Voting: Choosing the best model

In this exercise, you'll compare different classifiers and choose the one that performs the best.

Three individual classifiers have been fitted to the training set:

- clf_lr is a logistic regression.
- clf_dt is a decision tree.
- clf_knn is a 5-nearest neighbors classifier.

In [0]:
# Predict the labels of the test set
pred_lr = clf_lr.predict(X_test)
pred_dt = clf_dt.predict(X_test)
pred_knn = clf_knn.predict(X_test)

In [0]:
# Evaluate the performance of each model
score_lr = f1_score(y_test, pred_lr)
score_dt = f1_score(y_test, pred_dt)
score_knn = f1_score(y_test, pred_knn)

# Print the scores
print(score_lr)
print(score_dt)
print(score_knn)

#### Assembling your first ensemble

Your job is to leverage the voting ensemble technique using the sklearn API. It's up to you to instantiate the individual models and pass them as parameters to build your first voting classifier.

In [0]:
# Instantiate the individual models
clf_knn = KNeighborsClassifier(n_neighbors=5)
clf_lr = LogisticRegression(class_weight='balanced')
clf_dt = DecisionTreeClassifier(min_samples_leaf=3, min_samples_split=9, random_state=500)

# Create and fit the voting classifier
clf_vote = VotingClassifier(
    estimators=[('knn', clf_knn), ('lr', clf_lr), ('dt', clf_dt)]
)
clf_vote.fit(X_train, y_train)

#### Evaluating your ensemble

Remember to use f1_score() to evaluate the performance. In addition, you'll create a classification report on the test set (X_test, y_test) using the classifiation_report() function.

In [0]:
# Calculate the predictions using the voting classifier
pred_vote = clf_vote.predict(X_test)

# Calculate the F1-Score of the voting classifier
score_vote = f1_score(y_test, pred_vote)
print('F1-Score: {:.3f}'.format(score_vote))

# Calculate the classification report
report = classification_report(y_test, pred_vote)
print(report)

**Observation:**

The performance is not higher, but is at least as good as the best individual model - the decision tree. This may be because the other two models had a performance around 10% lower than the decision tree.

### Averaging

#### Predicting GoT deaths

Let's now build an ensemble model using the averaging technique. The following individual models have been built:

Logistic Regression (clf_lr).
Decision Tree (clf_dt).
Support Vector Machine (clf_svm).
As the target is binary, all these models might have good individual performance. Your objective is to combine them using averaging. Recall from the video that this is the same as a soft voting approach, so you should still use the VotingClassifier().

In [0]:
# Build the individual models
clf_lr = LogisticRegression(class_weight='balanced')
clf_dt = DecisionTreeClassifier(min_samples_leaf=3, min_samples_split=9, random_state=500)
clf_svm = SVC(probability=True, class_weight='balanced', random_state=500)

# List of (string, estimator) tuples
estimators = [('lr', clf_lr), ('dt', clf_dt), ('svm', clf_svm)]

# Build and fit an averaging classifier
clf_avg = VotingClassifier(estimators, voting='soft')
clf_avg.fit(X_train, y_train)

# Evaluate model performance
acc_avg = accuracy_score(y_test,  clf_avg.predict(X_test))
print('Accuracy: {:.2f}'.format(acc_avg))

#### Soft vs. hard voting

Three individual classifiers have been instantiated for you:

- A DecisionTreeClassifier (clf_dt).
- A LogisticRegression (clf_lr).
-A KNeighborsClassifier (clf_knn).

Your task is to try both voting and averaging to determine which is better.

In [0]:
# List of (string, estimator) tuples
estimators = [('dt', clf_dt), ('lr', clf_lr), ('knn', clf_knn)]

# Build and fit a voting classifier
clf_vote = VotingClassifier(estimators)
clf_vote.fit(X_train, y_train)

# Build and fit an averaging classifier
clf_avg = VotingClassifier(estimators, voting='soft')
clf_avg.fit(X_train, y_train)

# Evaluate the performance of both models
acc_vote = accuracy_score(y_test, clf_vote.predict(X_test))
acc_avg = accuracy_score(y_test,  clf_avg.predict(X_test))
print('Voting: {:.2f}, Averaging: {:.2f}'.format(acc_vote, acc_avg))

## 2. Bagging

Bagging is the ensemble method behind powerful machine learning algorithms such as random forests. In this chapter you'll learn the theory behind this technique and build your own bagging models using scikit-learn.

### The strength of “weak” models


#### Restricted and unrestricted decision trees

Here, you will build two separate decision tree classifiers. In the first, you will specify the parameters `min_sample_leaf` and `min_sample_split`, but not a maximum depth, so that the tree can fully develop without any restrictions.

In the second, you will specify some constraints by limiting the depth of the decision tree. By then comparing the two models, you'll better understand the notion of a "weak" learner.

In [0]:
# Build unrestricted decision tree
clf = DecisionTreeClassifier(min_samples_leaf=3, min_samples_split=9, random_state=500)
clf.fit(X_train, y_train)

# Predict the labels
pred = clf.predict(X_test)

# Print the confusion matrix
cm = confusion_matrix(y_test, pred)
print('Confusion matrix:\n', cm)

# Print the F1 score
score = f1_score(y_test, pred)
print('F1-Score: {:.3f}'.format(score))

In [0]:
# Build restricted decision tree
clf = DecisionTreeClassifier(max_depth=4, max_features=2, random_state=500)
clf.fit(X_train, y_train)

# Predict the labels
pred = clf.predict(X_test)

# Print the confusion matrix
cm = confusion_matrix(y_test, pred)
print('Confusion matrix:\n', cm)

# Print the F1 score
score = f1_score(y_test, pred)
print('F1-Score: {:.3f}'.format(score))

**Observation:**

Model A is a fine-tuned decision tree, with a decent performance on its own. Model B is 'weak', restricted in height and with performance just above 50%.

### Bootstrap aggregating


#### Training with bootstrapping

Let's now build a "weak" decision tree classifier and train it on a sample of the training set drawn with replacement. This will help you understand what happens on every iteration of a bagging ensemble.

In [0]:
# Take a sample with replacement
X_train_sample = X_train.sample(frac=1.0, replace=True, random_state=42)
y_train_sample = y_train.loc[X_train_sample.index]

# Build a "weak" Decision Tree classifier
clf = DecisionTreeClassifier(max_depth=4, max_features=2, random_state=500)

# Fit the model to the training sample
clf.fit(X_train_sample, y_train_sample)

#### A first attempt at bagging

You've seen what happens in a single iteration of a bagging ensemble. Now let's build a custom bagging model!

Two functions have been prepared for you:


`def build_decision_tree(X_train, y_train, random_state=None):`
    # Takes a sample with replacement,
    # builds a "weak" decision tree,
    # and fits it to the train set


`def predict_voting(classifiers, X):`
    # Makes the individual predictions 
    # and then combines them using "Voting"

In [0]:
def build_decision_tree(X_train, y_train, random_state=None):
        # Take a sample with replacement
        X_train_sample = X_train.sample(frac=1.0, replace=True, random_state=random_state)
        y_train_sample = y_train.loc[X_train_sample.index]

        # Build a "weak" Decision Tree classifier
        clf = DecisionTreeClassifier(max_depth=4, max_features=2, random_state=500)

        # Fit the model on the training sample
        clf.fit(X_train_sample, y_train_sample)
        
        return clf

In [0]:
def predict_voting(classifiers, X):
        # Make the individual predictions
        pred_list = [clf.predict(X) for clf in classifiers]
        
        # Combine the predictions using "Voting"
        pred_vote = []
        for i in range(X.shape[0]):
                individual_preds = np.array([pred[i] for pred in pred_list])
                combined_pred = stats.mode(individual_preds)[0][0]
                pred_vote.insert(i, combined_pred)
        
        return pred_vote


In [0]:
# Build the list of individual models
clf_list = []
for i in range(21):
	clf_list.append(build_decision_tree(X_train, y_train, random_state=i))

# Predict on the test set
pred = predict_voting(clf_list, X_test)

# Print the F1 score
print('F1 score: {:.3f}'.format(f1_score(y_test, pred)))

#### Bagging: the scikit-learn way

Let's now apply scikit-learn's BaggingClassifier to the Pokémon dataset.

You obtained an F1 score of around 0.56 with your custom bagging ensemble.

Will `BaggingClassifier()` beat it? Time to find out!

In [0]:
# Instantiate the base model
clf_dt = DecisionTreeClassifier(max_depth=4)

# Build and train the Bagging classifier
clf_bag = BaggingClassifier(
  base_estimator=clf_dt,
  n_estimators=21,
  random_state=500)
clf_bag.fit(X_train, y_train)

# Predict the labels of the test set
pred = clf_bag.predict(X_test)

# Show the F1-score
print('F1-Score: {:.3f}'.format(f1_score(y_test, pred)))

#### Checking the out-of-bag score

So far you've used the F1 score to measure performance. However, in this exercise you should use the accuracy score so that you can easily compare it to the out-of-bag score.

The decision tree classifier from the previous exercise, clf_dt, is available in your workspace.

In [0]:
# Build and train the bagging classifier
clf_bag = BaggingClassifier(
  base_estimator=clf_dt,
  n_estimators=21,
  oob_score=True,
  random_state=500)
clf_bag.fit(X_train, y_train)

# Print the out-of-bag score
print('OOB-Score: {:.3f}'.format(clf_bag.oob_score_))

# Evaluate the performance on the test set to compare
pred = clf_bag.predict(X_test)
print('Accuracy: {:.3f}'.format(accuracy_score(y_test, pred)))

### Bagging parameters: tips and tricks


#### Exploring the UCI SECOM data

It seems like this target is imbalanced, as more than 90% of the tests are negative. An individual model may be prone to overfitting, so it's a good idea to leverage an ensemble method here.

#### A more complex bagging model

The preprocessed dataset is available in your workspace as uci_secom, and training and test sets have been created for you.

As the target has a high class imbalance, use a "balanced" logistic regression as the base estimator here.

- Instantiate a balanced logistic regression to use as the base classifier by specifying the parameter class_weight = 'balanced'.
- Build a bagging classifier using the logistic regression as the base estimator, including the out-of-bag score, and using the maximum number of features as 10.
- Print the out-of-bag score to compare to the accuracy.

In [0]:
# Build a balanced logistic regression
clf_lr = LogisticRegression(class_weight='balanced')

# Build and fit a bagging classifier
clf_bag = BaggingClassifier(base_estimator=clf_lr, max_features=10, oob_score=True, random_state=500)
clf_bag.fit(X_train, y_train)

# Evaluate the accuracy on the test set and show the out-of-bag score
pred = clf_bag.predict(X_test)
print('Accuracy:  {:.2f}'.format(accuracy_score(y_test, pred)))
print('OOB-Score: {:.2f}'.format(clf_bag.oob_score_))

# Print the confusion matrix
print(confusion_matrix(y_test, pred))

#### Tuning bagging hyperparameters

In this exercise, let's see if we can improve model performance by modifying the parameters of the bagging classifier.

- Build a bagging classifier with 500 base estimators, 10 maximum features, and 0.65 (65%) maximum samples (max_samples). Sample without replacement.
- Use clf_bag to predict the labels of the test set, X_test.

In [0]:
# Build a balanced logistic regression
clf_base = LogisticRegression(class_weight='balanced', random_state=42)

# Build and fit a bagging classifier with custom parameters
clf_bag = BaggingClassifier(base_estimator=clf_base, n_estimators=500, max_samples=0.65, max_features=10, bootstrap=False, random_state=500)
clf_bag.fit(X_train, y_train)

# Calculate predictions and evaluate the accuracy on the test set
y_pred = clf_bag.predict(X_test)
print('Accuracy:  {:.2f}'.format(accuracy_score(y_test, y_pred)))

# Print the classification report
print(classification_report(y_test, y_pred))

## 3. Boosting

Boosting is class of ensemble learning algorithms that includes award-winning models such as AdaBoost. In this chapter, you'll learn about this award-winning model, and use it to predict the revenue of award-winning movies! You'll also learn about gradient boosting algorithms such as CatBoost and XGBoost.

### The effectiveness of gradual learning


#### Introducing the movie database

The dataset is loaded and available to you as movies.

Your main objective is to predict movie revenue - more specifically, log-revenue, which is the normalized version of the revenue feature.

Use the .describe() method to explore this feature. 

In [0]:
movies.describe()

#### Predicting movie revenue
Let's begin the challenge of predicting movie revenue by building a simple linear regression to predict the log-revenue of movies based on the 'budget' feature. The metric you will use here is the RMSE (root mean squared error). To calculate this using scikit-learn, you can use the mean_squared_error() function from the sklearn.metrics class and then take its square root using numpy.

The movies dataset has been loaded for you and split into train and test sets.

In [0]:
# Build and fit linear regression model
reg_lm = LinearRegression(normalize=True)
reg_lm.fit(X_train, y_train)

# Calculate the predictions on the test set
pred = reg_lm.predict(X_test)

# Evaluate the performance using the RMSE
rmse = np.sqrt(mean_squared_error(y_test, pred))
print('RMSE: {:.3f}'.format(rmse))

#### Boosting for predicted revenue

You'll build another linear regression, but this time the target values are the errors from the base model, calculated as follows:

`y_train_error = pred_train - y_train`

`y_test_error = pred_test - y_test`

In [0]:
# Fit a linear regression model to the previous errors
reg_error = LinearRegression(normalize=True)
reg_error.fit(X_train_pop, y_train_error)

# Calculate the predicted errors on the test set
pred_error = reg_error.predict(X_test_pop)

# Evaluate the updated performance
rmse_error = np.sqrt(mean_squared_error(y_test_error, pred_error))
print('RMSE: {:.3f}'.format(rmse_error))

### Adaptive boosting: award winning model


#### Your first AdaBoost model

In this exercise, you'll build your first AdaBoost model - an AdaBoostRegressor - in an attempt to improve performance even further.

In [0]:
# Instantiate a normalized linear regression model
reg_lm = LinearRegression(normalize=True)

# Build and fit an AdaBoost regressor
reg_ada = AdaBoostRegressor(base_estimator=reg_lm, n_estimators=12, random_state=500)
reg_ada.fit(X_train, y_train)

# Calculate the predictions on the test set
pred = reg_ada.predict(X_test)

# Evaluate the performance using the RMSE
rmse = np.sqrt(mean_squared_error(y_test, pred))
print('RMSE: {:.3f}'.format(rmse))

#### Tree-based AdaBoost regression

AdaBoost models are usually built with decision trees as the base estimators. 

We'll use twelve estimators as before to have a fair comparison. There's no need to instantiate the decision tree as it is the base estimator by default.

In [0]:
# Build and fit a tree-based AdaBoost regressor
reg_ada = AdaBoostRegressor(n_estimators=12, random_state=500)
reg_ada.fit(X_train, y_train)

# Calculate the predictions on the test set
pred = reg_ada.predict(X_test)

# Evaluate the performance using the RMSE
rmse = np.sqrt(mean_squared_error(y_test, pred))
print('RMSE: {:.3f}'.format(rmse))

#### Making the most of AdaBoost

As you have seen, for predicting movie revenue, AdaBoost gives the best results with decision trees as the base estimator.

In this exercise, you'll specify some parameters to extract even more performance. In particular, you'll use a lower learning rate to have a smoother update of the hyperparameters. Therefore, the number of estimators should increase. 

In [0]:
# Build and fit an AdaBoost regressor
reg_ada = AdaBoostRegressor(n_estimators=100, learning_rate=0.01, random_state=500)
reg_ada.fit(X_train, y_train)

# Calculate the predictions on the test set
pred = reg_ada.predict(X_test)

# Evaluate the performance using the RMSE
rmse = np.sqrt(mean_squared_error(y_test, pred))
print('RMSE: {:.3f}'.format(rmse))

### Gradient boosting


#### Sentiment analysis with GBM

Let's now use scikit-learn's GradientBoostingClassifier on the reviews dataset to predict the sentiment of a review given its text.

We will not pass the raw text as input for the model. The following pre-processing has been done for you:

1. Remove reviews with missing values.
2. Select data from the top 5 apps.
3. Select a random subsample of 500 reviews.
4. Remove "stop words" from the reviews.
5. Transform the reviews into a matrix, in which each feature represents the frequency of a word in a review.

In [0]:
# Build and fit a Gradient Boosting classifier
clf_gbm = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=500)
clf_gbm.fit(X_train, y_train)

# Calculate the predictions on the test set
pred = clf_gbm.predict(X_test)

# Evaluate the performance based on the accuracy
acc = accuracy_score(y_test, pred)
print('Accuracy: {:.3f}'.format(acc))

# Get and show the Confusion Matrix
cm = confusion_matrix(y_test, pred)
print(cm)

### Gradient boosting flavors


#### Movie revenue prediction with CatBoost

Let's finish up this chapter on boosting by returning to the movies dataset! In this exercise, you'll build a CatBoostRegressor to predict the log-revenue.

In [0]:
# Build and fit a CatBoost regressor
reg_cat = cb.CatBoostRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=500)
reg_cat.fit(X_train, y_train)

# Calculate the predictions on the set set
pred = reg_cat.predict(X_test)

# Evaluate the performance using the RMSE
rmse_cat = np.sqrt(mean_squared_error(y_test, pred))
print('RMSE (CatBoost): {:.3f}'.format(rmse_cat))	

#### Boosting contest: Light vs Extreme

CatBoost is highly recommended when there are categorical features. In this case, all features are numeric, therefore one of the other approaches might perform better.

In [0]:
# Build and fit a XGBoost regressor
reg_xgb = xgb.XGBRegressor(max_depth=3, learning_rate=0.1, n_estimators=100, random_state=500)
reg_xgb.fit(X_train, y_train)

# Build and fit a LightGBM regressor
reg_lgb = lgb.LGBMRegressor(max_depth=3, learning_rate=0.1, n_estimators=100, seed=500)
reg_lgb.fit(X_train, y_train)

# Calculate the predictions and evaluate both regressors
pred_xgb = reg_xgb.predict(X_test)
rmse_xgb = np.sqrt(mean_squared_error(y_test, pred_xgb))
pred_lgb = reg_lgb.predict(X_test)
rmse_lgb = np.sqrt(mean_squared_error(y_test, pred_lgb))

print('Extreme: {:.3f}, Light: {:.3f}'.format(rmse_xgb, rmse_lgb))

## 4. Stacking

Get ready to see how things stack up! In this final chapter you'll learn about the stacking ensemble method. You'll learn how to implement it from scratch as well as using the `mlxtend` library! You'll apply stacking to predict the edibility of North American mushrooms, and revisit the ratings of Google apps with this more advanced approach.

### The intuition behind stacking


#### Predicting mushroom edibility

Now that you have explored the data, it's time to build a first model to predict mushroom edibility.

The dataset is available to you as mushrooms. As both the features and the target are categorical, these have been transformed into "dummy" binary variables for you.

Let's begin with Naive Bayes (using scikit-learn's GaussianNB) and see how this algorithm performs on this problem.

In [0]:
# Instantiate a Naive Bayes classifier
clf_nb = GaussianNB()

# Fit the model to the training set
clf_nb.fit(X_train, y_train)

# Calculate the predictions on the test set
pred = clf_nb.predict(X_test)

# Evaluate the performance using the accuracy score
print("Accuracy: {:0.4f}".format(accuracy_score(y_test, pred)))

#### K-nearest neighbors for mushrooms

The Gaussian Naive Bayes classifier did a really good job for being an initial model. Let's now build a new model to compare it against the Naive Bayes.

In this case, the algorithm to use is a 5-nearest neighbors classifier. As the dummy features create a high-dimensional dataset, use the Ball Tree algorithm to make the model faster. 

In [0]:
# Instantiate a 5-nearest neighbors classifier
clf_knn = KNeighborsClassifier(n_neighbors=5, algorithm='ball_tree')

# Fit the model to the training set
clf_knn.fit(X_train, y_train)

# Calculate the predictions on the test set
pred = clf_knn.predict(X_test)

# Evaluate the performance using the accuracy score
print("Accuracy: {:0.4f}".format(accuracy_score(y_test, pred)))

### Build your first stacked ensemble


#### Applying stacking to predict app ratings

In this exercise you'll start building your first Stacking ensemble from scratch. 

In [0]:
# Build and fit a Decision Tree classifier
clf_dt = DecisionTreeClassifier(min_samples_leaf=3, min_samples_split=9, random_state=500)
clf_dt.fit(X_train, y_train)

# Build and fit a 5-nearest neighbors classifier using the 'Ball-Tree' algorithm
clf_knn = KNeighborsClassifier(n_neighbors=5, algorithm='ball_tree')
clf_knn.fit(X_train, y_train)

# Evaluate the performance using the accuracy score
print('Decision Tree: {:0.4f}'.format(accuracy_score(y_test, clf_dt.predict(X_test))))
print('5-Nearest Neighbors: {:0.4f}'.format(accuracy_score(y_test, clf_knn.predict(X_test))))

#### Building the second-layer classifier

Now you'll work on the next two steps of the process.

**Step 3**: Append the predictions to the dataset The predictions with the first-layer estimators were already calculated and are available to you as: pred_dt and pred_knn. You'll use these to create a DataFrame and append them to the training features.

**Step 4**: Build the second-layer meta estimator For this purpose you'll build the default DecisionTreeClassifier. This will take as input feature the concatenation of the original training features and the predictions from the first-layer estimators.

In [0]:
# Create a Pandas DataFrame with the predictions
pred_df = pd.DataFrame({
	'pred_dt': pred_dt,
    'pred_knn': pred_knn
}, index=X_train.index)

# Concatenate X_train with the predictions DataFrame
X_train_2nd = pd.concat([X_train, pred_df], axis=1)

# Build the second-layer meta estimator
clf_stack = DecisionTreeClassifier(random_state=500)
clf_stack.fit(X_train_2nd, y_train)

#### Stacked predictions for app ratings

Once the second-layer estimator is built you are ready for step 5: use the stacked ensemble for predictions.

In [0]:
# Create a Pandas DataFrame with the predictions
pred_df = pd.DataFrame({
	'pred_dt': pred_dt,
    'pred_knn': pred_knn
}, index=X_test.index)

# Concatenate X_test with the predictions DataFrame
X_test_2nd = pd.concat([X_test, pred_df], axis=1)

# Obtain the final predictions from the second-layer estimator
pred_stack = clf_stack.predict(X_test_2nd)

# Evaluate the new performance on the test set
print('Accuracy: {:0.4f}'.format(accuracy_score(y_test, pred_stack)))

#### A first attempt with mlxtend

You'll continue using the app ratings dataset. As you have already built a stacked ensemble model from scratch, you have a basis to compare with the model you'll now build with mlxtend.

In [0]:
# Instantiate the first-layer classifiers
clf_dt = DecisionTreeClassifier(min_samples_leaf=3, min_samples_split=9, random_state=500)
clf_knn = KNeighborsClassifier(n_neighbors=5, algorithm='ball_tree')

# Instantiate the second-layer meta classifier
clf_meta = DecisionTreeClassifier(random_state=500)

# Build the Stacking classifier
clf_stack = StackingClassifier(classifiers=[clf_dt, clf_knn], meta_classifier=clf_meta, use_features_in_secondary=True)
clf_stack.fit(X_train, y_train)

# Evaluate the performance of the Stacking classifier
pred_stack = clf_stack.predict(X_test)
print("Accuracy: {:0.4f}".format(accuracy_score(y_test, pred_stack)))

#### Back to regression with stacking

In Chapter 1, we treated the app ratings as a regression problem, predicting the rating on the interval from 1 to 5. 

In [0]:
# Instantiate the 1st-layer regressors
reg_dt = DecisionTreeRegressor(min_samples_leaf=11, min_samples_split=33, random_state=500)
reg_lr = LinearRegression(normalize=True)
reg_ridge = Ridge(random_state=500)

# Instantiate the 2nd-layer regressor
reg_meta = LinearRegression()

# Build the Stacking regressor
reg_stack = StackingRegressor(regressors=[reg_dt, reg_lr, reg_ridge], meta_regressor=reg_meta)
reg_stack.fit(X_train, y_train)

# Evaluate the performance on the test set using the MAE metric
pred = reg_stack.predict(X_test)
print('MAE: {:.3f}'.format(mean_absolute_error(y_test, pred)))

#### Mushrooms: a matter of life or death

You'll try the stacking classifier to see if the score can be improved. 

As stacking uses a meta-estimator (second layer classifier) which attempts to correct predictions from the first layer, some of the misclassified instances could be corrected. This is a very important problem, as the edibility of a mushroom is a matter of life or death.

In [0]:
# Create the first-layer models
clf_knn = KNeighborsClassifier(n_neighbors=5, algorithm='ball_tree')
clf_dt = DecisionTreeClassifier(min_samples_leaf=5, min_samples_split=15, random_state=500)
clf_nb = GaussianNB()

# Create the second-layer model (meta-model)
clf_lr = LogisticRegression()

# Create and fit the stacked model
clf_stack = StackingClassifier(classifiers=[clf_knn, clf_dt, clf_nb], meta_classifier=clf_lr)
clf_stack.fit(X_train, y_train)

# Evaluate the stacked model’s performance
print("Accuracy: {:0.4f}".format(accuracy_score(y_test, clf_stack.predict(X_test))))